# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [ ]:
# Data loading functions. Uncomment the one you want to use
# Import other classifiers you plan to use
from worclipo.load_data import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.decomposition import PCA


data = load_data()
print(f"The number of samples: {len(data.index)}")
print(f"The number of columns: {len(data.columns)}")
print(f"The number of liposarcoma in the dataset: {len(data[data['label'] == 'liposarcoma'])}")
print(f"The number of lipoma in the dataset: {len(data[data['label'] == 'lipoma'])}")

Now we know the number of samples, colums and labels. The next step is to look for missing data. 


In [ ]:
def split_data(data):

    # Separate features and target variable
    x = data.drop(['label'], axis=1)
    y = data['label']
    
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test


# Call the function
x_train, x_test, y_train, y_test = split_data(data)

# Optionally, print the sizes of the splits to verify
print("Training set size:", x_train.shape)
print("Testing set size:", x_test.shape)

In [ ]:
def find_zeros_in_data(data):
    # Maak een kopie van de data om de originele data niet te wijzigen
    data_copy = data.copy()
    
    # Vervang waarden die gelijk zijn aan 0 met NaN om ze gemakkelijk te tellen
    data_copy = data_copy.replace(0, np.nan)
    
    # Tel het aantal NaNs in elke kolom, wat overeenkomt met het originele aantal 0's
    zeros_count = data_copy.isna().sum()
    
    # Print het aantal 0's gevonden in elke kolom
    print("Aantal 0's per feature:\n", zeros_count)
    
    # Return het aantal 0's per feature voor verdere analyse indien nodig
    return zeros_count

# Roep de functie aan met je dataset 'data'
zeros_count = find_zeros_in_data(data)


Feature normalization

In [ ]:
# def scaling_data(x_train,x_test):
#     scaler = preprocessing.StandardScaler().fit(x_train)

#     # Applying scaler to train and test set
#     X_train_scaled = scaler.transform(x_train)
#     X_test_scaled = scaler.transform(x_test)

#     return X_train_scaled, X_test_scaled


Feature selection 

In [ ]:
# #Call the function to use X_train_scaled
# X_train_scaled = scaling_data(x_train,x_test)
# print(y_train.shape)
# print(y_train[1:10])

PCA

In [ ]:
# #desired_variance = 0.50  # 80% gewenste verklaarde variantie
# #pca = PCA(n_components=desired_variance)
# pca = PCA(n_components=2)

# #pca_X_train_scaled = pca.fit_transform(x)
# pca_X_train_scaled = pca.fit_transform(X_train_scaled)


# # Perform a PCA
# pca = PCA(n_components=2)
# pca.fit(X_train_scaled)
# X_train_pca = pca.transform(X_train_scaled)
# #X_test_pca = pca.transform(X_test_scaled)

# explained_variance_ratio = pca.explained_variance_ratio_
# total_explained_variance_ratio = explained_variance_ratio.sum()
# print(total_explained_variance_ratio)

# num_components = pca.n_components_
# print("Aantal componenten na PCA:", num_components)


# Scale the data to be normal
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
X_train_scaled = scaler.transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Perform a PCA
#pca = PCA(n_components=2)

desired_variance = 0.80 # 80% gewenste verklaarde variantie
pca = PCA(n_components=desired_variance)

pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

explained_variance_ratio = pca.explained_variance_ratio_
total_explained_variance_ratio = explained_variance_ratio.sum()
print(total_explained_variance_ratio)

num_components = pca.n_components_
print("Aantal componenten na PCA:", num_components)

sns.scatterplot(x=X_train_pca[:, 0], y=X_train_pca[:, 1], hue=y)
